# Init

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_path = 'data/PM_train.txt'
model_path = 'models/regression_model_v1.h5'
export_path = 'models/turbofan'
sequence_length = 50
nb_features = 25
nb_out = 1

# LSTM model

In [ ]:
model = Sequential()
model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae'])


Load model

In [ ]:
model.load_weights(model_path)

Test model

In [ ]:
# Load
def gen_sequence(id_df, seq_length, seq_cols):

    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]

    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]
        
def gen_labels(id_df, seq_length, label):

    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    return data_matrix[seq_length:num_elements, :]

columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

train_df = pd.read_csv(train_path, sep=' ', header=None)
train_df.drop(train_df[[26, 27]], axis=1, inplace=True)
train_df.columns = columns
train_df.sort_values(['id', 'cycle'], inplace=True)


# Preprocess
train_rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
train_rul.columns = ['id', 'max']
train_df = train_df.merge(train_rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)

# Extract test
train_df_1 = train_df[train_df['id'] == 1]
train_df_1 = train_df_1[sequence_cols]
len_train_df_1 = len(train_df_1)
last_test = [train_df_1[start:stop].values for start, stop in zip(range(0, len_train_df_1 - sequence_length), range(sequence_length, len_train_df_1))]
last_test_nd = np.array(last_test)
last_test_nd.shape

for i in last_test_nd:
    print(model.predict(np.reshape(i, (1, 50, 25))))

# Export

In [ ]:
model.save(export_path)

Test

In [ ]:
test_model = tf.keras.models.load_model(export_path)

for i in last_test_nd:
    print(test_model.predict(np.reshape(i, (1, 50, 25))))